In [3]:
import angr

# Demo: quick angr pipeline
**Goal:** show key angr features end-to-end:
1. load binary & project  
2. build CFG (fast)  
3. traverse CFG -> per-instruction disassembly (mnemonic + operands)  
4. inspect VEX IR for a block  
5. tiny symbolic execution: set symbolic input & find target


In [ ]:
# the path for the binary to be analyzed
BINARY_PATH = "./bound_vuln_stripped" 

## 1) Load an angr project & some quick checks:

In [9]:
proj_stripped = angr.Project(BINARY_PATH, auto_load_libs=False)
print("Arch:", proj_stripped.arch)
print("Entry:", hex(proj_stripped.entry))
print("Base:", hex(proj_stripped.loader.main_object.mapped_base))
print("Sections:", [ (s.name, hex(s.min_addr), hex(s.max_addr)) for s in proj_stripped.loader.main_object.sections ])
syms_stripped = proj_stripped.loader.main_object.symbols

Exception: Not a valid binary file: 'data/bound_vuln_stripped'

In [ ]:
BINARY_PATH = "./bound_vuln"
proj = angr.Project(BINARY_PATH, auto_load_libs=False)
section_syms = [s.name for s in proj_stripped.loader.main_object.sections]
section = [s.name for s in proj.loader.main_object.sections]
print(set(section) - set(section_syms))
main_symbol = proj.loader.main_object.get_symbol("main")
print (hex(main_symbol.rebased_addr))
main_symbol = proj.loader.main_object.symbols_by_addr[int('0x401120', 16)]
print(main_symbol.name)

## 2) Build a CFG (fast)

In [ ]:
cfg = proj.analyses.CFGFast(normalize=True)
funcs = list(cfg.kb.functions.items())
for addr, func in funcs:
    print(f"  {func.name:40} -> {hex(addr)}")
if "main" in cfg.kb.functions:
    main_addr = cfg.kb.functions["main"].addr
    print(hex(main_addr))

## 3) Traverse the CFG: iterate blocks in `main`, extract disasm, opcode & operands
**For each block**: we get capstone disassembly if available (`block.capstone`) and per-insn fields:
- `insn.mnemonic` and `insn.op_str`  
- raw bytes: `insn.bytes`

In [ ]:
for func_addr, func in cfg.functions.items():
    if func.name.startswith('sub_') or func.name in ['UnresolvableCallTarget', 'UnresolvableJumpTarget']:
        continue
    print(func.name)
    if(func.name!="main"):
        continue
    for block in func.blocks:
        block_addr = block.addr
        print(block_addr)
        disassembly = block.capstone.insns
        records = []
        for insn in disassembly:
            records.append({
                "addr": hex(insn.address),
                "mnemonic": insn.mnemonic,
                "op_str": insn.op_str,
                "bytes": insn.bytes.hex()
            })
        print(records)

## 4) Inspect VEX IR for a block
angr exposes PyVEX `IRSBB` via `block.vex` (or `block.vex.pp()` text)

In [ ]:
main_function = cfg.functions[main_addr]
for block in main_function.blocks:
    print("VEX IR (short):")
    print(block.vex.pp())

    # Or iterate VEX statements
    stmts = block.vex.statements
    print("\nStatements count:", len(stmts))
    for s in stmts:
        print(" -", s)  
    break

## 5) Simple data-flow: Symbolic Execution to trace the value in specific registers


In [ ]:
# define the hook for fgets
class MyFgets(angr.SimProcedure):
    def run(self, s, size, stream):
        data = b"secret\x00"
        bv = claripy.BVV(data)
        self.state.memory.store(s, bv)
        return s

# # Hook commonly used libc symbols
proj.hook_symbol("puts", angr.SIM_PROCEDURES['libc']['puts']())
proj.hook_symbol("strlen", angr.SIM_PROCEDURES['libc']['strlen']())
proj.hook_symbol("strcmp", angr.SIM_PROCEDURES['libc']['strcmp']())
# proj.hook_symbol("fgets", angr.SIM_PROCEDURES['libc']['fgets']())
proj.hook_symbol("fgets", MyFgets())
proj.hook_symbol("__memcpy_chk", angr.SIM_PROCEDURES['libc']['memcpy']())

# // bound_vuln.c
# #include <stdio.h>
# #include <string.h>

# int check(const char* in){
#     char buf[16];
#     if (strlen(in) <= 32) {
#         strcpy(buf, in);
#         return strcmp(buf, "secret");
#     }
#     return -1;
# }
# int main(){
#     char input[128];
#     puts("Enter token:");
#     fgets(input, sizeof(input), stdin);
#     input[strcspn(input, "\n")] = 0;
#     if (check(input)==0) puts("OK");
#     else puts("NO");
#     return 0;
# }

func = next(cfg.functions.get_by_name("main"))

# the entry for symbolic execution
state = proj.factory.blank_state(addr=func.addr)

# define rdi as a symbol
import claripy
rdi_sym = claripy.BVS("rdi_sym", 64)
state.regs.rdi = rdi_sym

# create the symbolic execution manager
simgr = proj.factory.simgr(state)

# execute step by step
for step in range(200):
    if len(simgr.active) == 0:
        print("No more active states.")
        break
    
    state = simgr.active[0]
    print("RDI =", state.solver.eval_upto(state.regs.rdi, 1))
    print("RDI (AST) =", state.regs.rdi)
    print(f"\n--- Step {step}, PC = {hex(state.addr)} ---")
    if((state.addr)==int("0x4011ab", 16)):
        print("hell no")
        break
    if((state.addr)==int("0x401181", 16)):
        print("Oh yes")
        break
    try:
        block = proj.factory.block(state.addr)
        print(block.disassembly)
    except:
        pass

    # one step further
    simgr.step()
